In [15]:
import paramiko
import os
import subprocess

# Define the necessary variables
username = "sanjeevc"
hostname = "perlmutter.nersc.gov"
private_key_path = "~/.ssh/nersc"
local_ports = [(8786, "10.249.5.133", 8786), (8787, "10.249.5.133", 8787)]

# Expand user directory if needed
private_key_path = os.path.expanduser(private_key_path)

# Create the SSH tunneling command
ssh_command = [
    "ssh",
    "-l", username,
    "-i", private_key_path,
    "-N"  # Do not execute remote commands, just keep the tunnel open
]

# Add port forwarding options
for local_port, remote_host, remote_port in local_ports:
    ssh_command.extend(["-L", f"{local_port}:{remote_host}:{remote_port}"])

# Add the hostname
ssh_command.append(f"{username}@{hostname}")

# Run the SSH command to create the tunnel
try:
    subprocess.run(ssh_command)
except Exception as e:
    print(f"Failed to establish SSH tunnel: {e}")


***************************************************************************
                          NOTICE TO USERS

Lawrence Berkeley National Laboratory operates this computer system under 
contract to the U.S. Department of Energy.  This computer system is the 
property of the United States Government and is for authorized use only.
Users (authorized or unauthorized) have no explicit or implicit 
expectation of privacy.

Any or all uses of this system and all files on this system may be
intercepted, monitored, recorded, copied, audited, inspected, and disclosed
to authorized site, Department of Energy, and law enforcement personnel,
as well as authorized officials of other agencies, both domestic and foreign.
By using this system, the user consents to such interception, monitoring,
recording, copying, auditing, inspection, and disclosure at the discretion
of authorized site or Department of Energy personnel.

Unauthorized or improper use of this system may result in administrative

KeyboardInterrupt: 

In [17]:
import paramiko
import os
import socket

# Define the necessary variables
username = "sanjeevc"
hostname = "perlmutter.nersc.gov"
private_key_path = "~/.ssh/nersc"
local_ports = [(8786, "10.249.5.133", 8786), (8787, "10.249.5.133", 8787)]

# Expand user directory if needed
private_key_path = os.path.expanduser(private_key_path)

# Load the SSH private key
key = paramiko.RSAKey(filename=private_key_path)

# Create an SSH client
client = paramiko.SSHClient()

# Automatically add the server's host key (not recommended for production)
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# Create port forwarding for each local port
def forward_tunnel(local_port, remote_host, remote_port, transport):
    # Create a socket to listen on the local port
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind(('localhost', local_port))
    sock.listen(5)
    
    def handler(channel, host, port):
        while True:
            client_socket, addr = sock.accept()
            forward_socket = transport.open_channel(
                'direct-tcpip', (remote_host, remote_port), client_socket.getpeername()
            )
            if forward_socket is None:
                print(f"Failed to open channel for {remote_host}:{remote_port}")
                client_socket.close()
                continue
            # Forward traffic between the client socket and the remote socket
            while True:
                data = client_socket.recv(1024)
                if not data:
                    break
                forward_socket.send(data)
                data = forward_socket.recv(1024)
                if not data:
                    break
                client_socket.send(data)
            forward_socket.close()
            client_socket.close()

    transport.request_port_forward('localhost', local_port, handler)

try:
    # Connect to the remote server
    client.connect(hostname, username=username, pkey=key)
    transport = client.get_transport()
    
    # Set up local port forwarding
    for local_port, remote_host, remote_port in local_ports:
        forward_tunnel(local_port, remote_host, remote_port, transport)
    
    # Keep the connection alive
    client.exec_command("sleep infinity")

except Exception as e:
    print(f"Failed to establish SSH tunnel: {e}")

finally:
    # Close the SSH connection
    client.close()


Failed to establish SSH tunnel: [Errno 48] Address already in use
